# Bug triage with Deep Learning - PROPOSE

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 40
MAX_SEQUENCE_LENGTH_D = 150 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'netbeans'
METHOD = 'propose'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = 'propose_feature@number_of_epochs@epochs_64batch({})'.format(DOMAIN)
SAVE_PATH_FEATURE = 'propose_feature_@number_of_epochs@epochs_64batch({})'.format(DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [9]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [10]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


216715

#### Dicionário de títulos e descrições

In [11]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 34.3 s, sys: 1.53 s, total: 35.8 s
Wall time: 35.8 s


#### Hashing bugs by buckets

In [12]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [13]:
experiment.prepare_dataset(issues_by_buckets)
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids


In [14]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '1\n',
 'bug_status': '0\n',
 'component': '246\n',
 'creation_ts': '2004-05-12 16:16:00 +0000',
 'delta_ts': '2004-08-13 12:11:54 +0000',
 'description': 'copy correct solution for batching from search law occurred in request processor java law at java util linked list list itr check for artwork linked list java at java util linked list list itr next linked list java at java util collections next collections java at org openide nodes organization update artwork java at org openide nodes person set artwork at org netbeans modules tasklist core artwork task artwork run artwork java at org openide util task run task java at org openide util request product run request processor java catch at org openide util request processor processor run request processor java',
 'description_word': array([ 392,  679, 1562,   29,    1,   61,  409,   60,  229,   14,   69,
          63,    2,   60,    6,    2,   41, 1473,  144,  144, 2500,  253,
          29,   13, 1473,  144,    2,    6

### Generating the batch test

In [15]:
"Train ", len(baseline.dup_sets_train)

('Train ', 32276)

In [16]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = baseline.batch_iterator(baseline.train_data, 
                                                                                          baseline.dup_sets_train, 
                                                                                          batch_size_test, 1)
test_gen = ([valid_input_sample['title'], valid_input_pos['title'], valid_input_neg['title'], 
             valid_input_sample['description'], valid_input_pos['description'], valid_input_neg['description'],
            valid_input_sample['info'], valid_input_pos['info'], valid_input_neg['info']], valid_sim)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 75.9 ms, sys: 0 ns, total: 75.9 ms
Wall time: 75.2 ms


In [17]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 150), (128, 544), (128,))

### Validar entrada

In [18]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, 5)

***Title***: product opening project
***Title***: strange autocompletion
***Description***: all the testing i did went smoothly and i really didn t need to use the mouse for all the time the only problem i discovered is if you try to use product while browsig your computer e g opening the project without the mouse you will notice user unfriendly interface i would expect the organization key to switch between the parts of the browsing window instead of completing path of the listed directories until now i really don t know how to swich between them not coincidentally
***Description***: using only keyboard when adding project in organization dialog pressing organization to change the focused component but the selected component organization field is still focused each pressing of organization tries to autocomplete the path in this component that is really strange the same functionality is mapped to the right arrow key as well well the autocompletion is a usefull thing but so implemented 

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [19]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed_fasttext.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [20]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 122604'

In [21]:
import io

def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'crawl-300d-2M.vec')
    f = open(embed_path, 'rb')
    f = io.open(embed_path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, f.readline().split())

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed_fasttext.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading FastText")
    for line in loop:
        tokens = line.rstrip().split(' ')
        embed = list(map(float, tokens[1:]))
        word = tokens[0]
        embeddings_index[word] = np.asarray(embed, dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in FastText 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [22]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1999995 word vectors in FastText 42B 300d.



CPU times: user 2min 11s, sys: 4.8 s, total: 2min 16s
Wall time: 2min 12s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [23]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [24]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer',
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### ARCII

In [25]:
from keras.layers.core import Dense, Reshape, Flatten, Dropout
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

def arcii_model(embedding_layer, max_sequence_length):
    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)
    
    filters_1d=max_sequence_length
    kernel_size_1d=3
    num_conv2d_layers=2
    filters_2d=[32,16]
    kernel_size_2d=[[3,3], [3,3]]
    mpool_size_2d=[[2,2], [2,2]]
    dropout_rate=0.5
    
    layer1_conv=Conv1D(filters=filters_1d, kernel_size=kernel_size_1d, padding='same')(embedded_sequences)
    layer1_activation=Activation('relu')(layer1_conv)
    layer1_reshaped=Reshape((max_sequence_length, max_sequence_length, -1))(layer1_activation)
    z=MaxPooling2D(pool_size=(2,2))(layer1_reshaped)

    for i in range(num_conv2d_layers):
        z=Conv2D(filters=filters_2d[i], kernel_size=kernel_size_2d[i], padding='same')(z)
        z=Activation('tanh')(z)
        z=MaxPooling2D(pool_size=(mpool_size_2d[i][0], mpool_size_2d[i][1]))(z)

    pool1_flat=Flatten()(z)
    pool1_flat_drop=Dropout(rate=dropout_rate)(pool1_flat)
    pool1_norm=BatchNormalization()(pool1_flat_drop)
    mlp1=Dense(300)(pool1_norm)
    output=Activation('tanh')(mlp1)
    feature_model = Model(inputs=[sequence_input], outputs=[output], name = 'FeatureARCIIGenerationModel') # inputs=visible
    return feature_model

### CNN Dilated

In [26]:
from keras.constraints import max_norm
import math

def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def block(block_input):        
        residual =    block_input
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False)(block_input) #kernel_regularizer=l2(l2_layer_reg)                    
        
        activation_out = Activation('tanh')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False)(activation_out) # use_bias=False, kernel_constraint=max_norm(1.)
        
        c1x1_out =    Conv1D(1,1, activation='linear', use_bias=False)(activation_out)
                      
        block_out =   Add()([residual, c1x1_out])
        
        return block_out, skip_out
    return block

def cnn_dilated_model(embedding_layer, max_sequence_length):
    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    units = 128
    number_of_layers = 6

    # Embedding layer with CNN dilated
    #la, lb = DC_CNN_Block(units,2,1,0.01)(embedded_sequences)
    la = embedded_sequences
    attention_layes = []
    filters_size = [3, 4, 5]
    number_of_filters = len(filters_size)
    for index in range(1, number_of_layers + 1):
        la, lb = DC_CNN_Block(units, 5, int(math.pow(2, index)), 0.01)(la)
        #la = Dropout(.90)(la)
        #lb = Dropout(.90)(lb)
        attention_layes.append(lb)

#     l1a, l1b = DC_CNN_Block(units, 3, 2, 0.01)(la)
#     l2a, l2b = DC_CNN_Block(units, 3, 4, 0.01)(l1a)
#     l3a, l3b = DC_CNN_Block(units, 3, 8, 0.01)(l2a)

    attention_layer = Add()(attention_layes)
    # attention_layer =   Add()([l1a, l2a])
    
    #layer = Add()([attention_layer, l9])
    
    layer =   Activation('tanh')(attention_layer)

    #layer =  Conv1D(1,1, activation='linear', use_bias=False)(layer)
    
    #layer = Flatten()(layer)
    #layer = GlobalAveragePooling1D()(layer)
    #layer = Dropout(0.50)(layer)
    #layer = Dense(300, activation='tanh')(layer)
    layer = GRU(150, activation='tanh', return_sequences=False)(layer)

    cnn_dilated_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureCNNDilatedGenerationModel') # inputs=visible
    return cnn_dilated_feature_model

### CNN with filter 3,4,5

In [27]:
import keras
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, AveragePooling1D

def cnn_model(embedding_layer, max_sequence_length):

    sequence_input = Input(shape=(max_sequence_length,), name='Feature_BugInput')
    #sequence_input = Input(shape=(None,), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    # best combination filter (3, 4, 5) e 128 e 256
    convs = []
    filter_sizes = [3, 4, 5]
    n_filters = 32

    for index, filter_size in enumerate(filter_sizes):
        l_conv = Conv1D(filters=n_filters, kernel_size=filter_size, kernel_initializer='random_uniform',
                bias_initializer='zeros')(embedded_sequences)
        l_pool = AveragePooling1D(pool_size=filter_size)(l_conv) # index+1
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=n_filters * 3, kernel_size=5)(l_merge)
    layer = GlobalAveragePooling1D()(l_merge)
    #layer = Flatten()(l_merge)
    #layer = Dense(300, activation='tanh')(layer)
    #layer = GRU(100, activation='tanh', return_sequences=False)(l_merge)
    #layer = LeakyReLU()(layer)

    cnn_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureCNNGenerationModel') # inputs=visible

    return cnn_feature_model

### Bi-LSTM

In [28]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, GlobalAveragePooling1D

def lstm_model(embedding_layer, max_sequence_length):
    number_lstm_units = 50
    rate_drop_lstm = 0
    recurrent_dropout = 0

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    #sequence_input = Input(shape=(None, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Creating LSTM Encoder
    lstm_layer = Bidirectional(GRU(number_lstm_units, activation='tanh', 
                                   return_sequences=True, kernel_initializer='random_uniform',
                bias_initializer='zeros'), # dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm 
                               merge_mode='ave')

#     lstm_layer = LSTM(number_lstm_units, return_sequences=True, kernel_initializer='random_uniform',
#                 bias_initializer='zeros')(embedded_sequences)
#     layer = LSTM(number_lstm_units, return_sequences=False, kernel_initializer='random_uniform',
#                 bias_initializer='zeros')(lstm_layer)

    layer = lstm_layer(embedded_sequences)
    layer = GlobalAveragePooling1D()(layer)
    #layer = Dense(300, activation='tanh')(layer)
    #layer = GRU(100, activation='tanh', return_sequences=False)(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh', kernel_initializer='random_uniform',
                bias_initializer='zeros')(info_input)
    
    #layer = GRU(100, activation='tanh')(layer)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [30]:
from keras import backend as K
import tensorflow as tf

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=False)

def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

# https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
def triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def custom_margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

#### Propose

In [31]:
from keras.initializers import TruncatedNormal
from keras.regularizers import l2
from keras.layers import Average, Dot, Maximum, Permute

def residual_bug():
    def block(block_input):
        shape_size_cols = K.int_shape(block_input)[1]
        shape_size_rows = 1
        
        residual =  block_input
        residual = Activation('relu')(residual)
        #residual = BatchNormalization()(residual)
        
        layer_out = Reshape((shape_size_cols, shape_size_rows))(block_input)
        layer_out = GRU(100, activation='relu', return_sequences=True)(layer_out)
        #layer_out = GRU(100, activation='relu', return_sequences=True)(layer_out)
        #layer_out = Reshape((shape_size_cols, ))(layer_out)
        layer_out = GlobalAveragePooling1D()(layer_out)
        #layer_out = BatchNormalization()(layer_out)
        layer_out = Dense(50, activation='relu')(layer_out)
        #layer_out = BatchNormalization()(layer_out)
        layer_out = Dense(shape_size_cols, activation='relu', use_bias=True, kernel_initializer='random_uniform')(layer_out)
        skip_out = Dense(shape_size_cols, activation='relu', use_bias=True, kernel_initializer='random_uniform')(layer_out)
        #layer_out = Activation('relu')(layer_out)
        #layer_out = BatchNormalization()(layer_out)
        
        block_out = Add()([residual, layer_out])
        #block_out = Activation('relu')(block_out)
        return block_out, skip_out
    return block

In [32]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, Average
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    bug_i_feat = categorical_feature_model(bug_i)
    
#     encoded_t_1a, encoded_t_1b  = residual_bug()(bug_t_feat)
#     encoded_d_1a, encoded_d_1b  = residual_bug()(bug_d_feat)
#     bug_t_feat = encoded_t_1a
#     bug_d_feat = encoded_d_1a
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    #bug_feature_output, bug_feature_output_1b = residual_bug()(bug_feature_output)
    #bug_feature_output_1a = Dropout(.5)(bug_feature_output_1a)
    #bug_feature_output, bug_feature_output_2b = residual_bug()(bug_feature_output_1a)
    
    #bug_feature_output = Add()([bug_feature_output_1b, bug_feature_output_2b])
    #bug_feature_output = BatchNormalization()(bug_feature_output)
    #bug_feature_output = Activation('relu')(bug_feature_output)
#     bug_feature_output = Dropout(.75)(bug_feature_output)
#     shape_size = K.int_shape(bug_feature_output)[1]
#     bug_feature_output = Dense(shape_size, activation='linear', use_bias=False)(bug_feature_output)
#     bug_feature_output = Dropout(.33)(bug_feature_output)
#     bug_feature_output = Dense(100)(bug_feature_output)
    
    #bug_feature_output  = residual_bug()(bug_feature_output)
    #bug_feature_output = BatchNormalization()(bug_feature_output)
    #     encoded_2a, encoded_2b  = residual_bug()(encoded_1a)
    
    #     bug_feature_output = Add()([encoded_1b, encoded_2b])
    #     bug_feature_output = Activation('tanh')(bug_feature_output)
    
    # Bug representation layer
    # bug_feature_output = Dense(300, activation='tanh')(bug_feature_output)
    
    bug_feature_model = Model(inputs=[bug_t, bug_d, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [33]:
def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    
    # Distance
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance', output_shape=[1])([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance', output_shape=[1])([encoded_anchor, encoded_negative])

    # Loss function only works with a single output
    output = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances',
        output_shape=(2, 1)
    )([positive_d, negative_d])
    
    #loss = MarginLoss()(output)

    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer=optimizer, loss=custom_margin_loss, metrics=[pos_distance, neg_distance, custom_margin_loss])

    return similarity_model

In [ ]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False)
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False)

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
'''
desc_feature_model = cnn_dilated_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
title_feature_model = lstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = baseline.batch_iterator(baseline.train_data, baseline.dup_sets_train, batch_size, 1)
    train_batch = [train_input_sample['title'], train_input_sample['description'], train_input_sample['info'],
                   train_input_pos['title'], train_input_pos['description'], train_input_pos['info'], 
                   train_input_neg['title'], train_input_neg['description'], train_input_neg['info']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _ = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets)
        print("Epoch: {} Loss: {:.2f}, CustomLoss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, recall@25: {:.2f}".format(epoch+1,
                                                                                                         h[0],  h[3],
                                                                                                         h[1], h[2], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, CustomLoss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}".format(epoch+1,
                                                                                                         h[0],  h[3],
                                                                                                         h[1],
                                                                                                         h[2]))
    loss = h[3]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 544)          0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
info_pos (InputLayer)           (None, 544)          0                                            
__________________________________________________________________________________________________
title_pos 

In [ ]:
_[:20]

In [ ]:
recall, exported_rank = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets)

"recall@25 last epoch:", recall

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

#### Getting the model trained

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [ ]:
model.summary()

In [ ]:
recall, exported_rank = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets)

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report